In [ ]:
pip install transformers &> /dev/null

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = 'gpt2-medium'
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
import numpy as np
def score(sentence):
    tokens_tensor = tokenizer(sentence, return_tensors="pt")
    loss = model(**tokens_tensor, labels=tokens_tensor['input_ids'])[0]
    # Use detach().numpy().item() to convert tensor to float 
    return loss.detach().numpy().item() 

In [ ]:
def use_gpt2(question,options):
  scores = [score(question.replace("[MASK]", o)) for o in options]
  # # Uncomment this code to print score of each option
  # for i in zip(options,scores):
  #   print(i)
  return options[np.argmin(scores)]

In [ ]:
def compare(results, answers):
  correct = 0 
  for i in range(len(results)):
    if results[i] == answers[i]:
      correct += 1
    # # Uncomment this code to print wrong answer 
    # else:
    #   print(i)
    #   print(results[i])
    #   print(answers[i])
  return correct/len(results)

In [ ]:
import urllib.request, json 
def read_json(path):
  with urllib.request.urlopen(path) as url:
    data = json.loads(url.read().decode())
  questions = []
  answers = []
  options = []
  for d in data.values():
    questions.append(d['question'])
    answers.append(d['anwser'])
    option = [d[str(i)] for i in range(1,5)]
    options.append(option)
  return questions, answers, options

In [49]:
# Use this code to get words filling dataset
questions, answers, options = read_json("https://raw.githubusercontent.com/tranganhthuan/TOEIC_Mask_Filling/main/data/words_filling.json")

# # Use this code to get sentence filling dataset
# questions, answers, options = read_json("https://raw.githubusercontent.com/tranganhthuan/TOEIC_Mask_Filling/main/data/sentence_filling.json")

In [50]:
# Use this code to calculate result only
results = [use_gpt2(q,o) for (q,o) in zip(questions, options)]

# # Use this code to print the loop
# i = 0
# results = []
# for q,o in zip(questions, options):
#   print(i)
#   results.append(use_gpt2(q,o))
#   i += 1

In [51]:
compare(results, answers)

0.575